In [ ]:
import os
import sys
import unittest
from math import pi
import numpy as np
import logging
from itertools import permutations
from ddt import ddt, data
from numpy.testing import assert_allclose
import contextlib

from qiskit import transpile
from qiskit.exceptions import QiskitError
from qiskit.circuit import QuantumCircuit, QuantumRegister
from qiskit.providers.basicaer import QasmSimulatorPy
from qiskit.quantum_info.random import random_unitary, random_statevector, random_pauli
from qiskit.quantum_info.states import Statevector
from qiskit.circuit.library import QuantumVolume
from qiskit.providers.aer import AerSimulator
from qiskit.quantum_info.operators.operator import Operator
from qiskit.quantum_info.operators.symplectic import Pauli, SparsePauliOp
from qiskit.quantum_info.operators.predicates import matrix_equal
from qiskit.visualization.state_visualization import numbers_to_latex_terms, state_to_latex
from qiskit.circuit.library import QFT, HGate
from test.terra import common
from qiskit_aer.aererror import AerError
from qiskit_aer.quantum_info.states import AerStatevector

In [ ]:
def rand_vec(n, normalize=False):
    """Return complex vector or statevector"""
    seed = np.random.randint(0, np.iinfo(np.int32).max)
    rng = np.random.default_rng(seed)
    vec = rng.random(n) + 1j * rng.random(n)
    if normalize:
        vec /= np.sqrt(np.dot(vec, np.conj(vec)))
    return vec

def assertEqual(a, b):
    assert a == b

def assertNotEqual (a, b):   
    assert a != b

def assertTrue(a):
    assertEqual(a, True)

def assertFalse(a):
    assertEqual(a, False)

def assertIsNone(a):
    assertTrue(a is None)

def assertIn(a, b):
    assertTrue(a in b)

def assertRaises(exception, func, *args, **kwds):
    try:
        func(*args, **kwds)
    except exception:
        pass

def assertAlmostEqual(a, b):
    assertTrue(np.isclose(a, b))

# https://stackoverflow.com/questions/23549419/assert-that-two-dictionaries-are-almost-equal
def _assertDictAlmostEqual(dict1, dict2, rel_tol=1e-8):
    """
    If dictionary value is a number, then check that the numbers are almost equal, otherwise check if values are exactly equal
    Note: does not currently try converting strings to digits and comparing them. Does not care about ordering of keys in dictionaries
    Just returns true or false
    """
    if len(dict1) != len(dict2):
        return False
    # Loop through each item in the first dict and compare it to the second dict
    for key, item in dict1.items():
        # If it is a nested dictionary, need to call the function again
        if isinstance(item, dict):
            # If the nested dictionaries are not almost equal, return False
            if not _assertDictAlmostEqual(dict1[key], dict2[key], rel_tol=rel_tol):
                return False
        # If it's not a dictionary, then continue comparing
        # Put in else statement or else the nested dictionary will get compared twice and
        # On the second time will check for exactly equal and will fail
        else:
            # If the value is a number, check if they are approximately equal
            if isinstance(item, Number):
                # if not abs(dict1[key] - dict2[key]) <= rel_tol:
                # https://stackoverflow.com/questions/5595425/what-is-the-best-way-to-compare-floats-for-almost-equality-in-python
                if hasattr(item, 'imag'):
                    if not math.isclose(dict1[key].real, dict2[key].real, rel_tol=rel_tol) or not math.isclose(dict1[key].imag, dict2[key].imag, rel_tol=rel_tol):
                        return False
                else:
                    if not math.isclose(dict1[key], dict2[key], rel_tol=rel_tol):
                        return False
            else:
                if not (dict1[key] == dict2[key]):
                    return False
    return True

def assertDictAlmostEqual(dict1, dict2, rel_tol=1e-8):
    assertTrue(_assertDictAlmostEqual(dict1, dict2, rel_tol))

@contextlib.contextmanager
def subTest(**kwds):
    try:
        yield
    except Exception as e:
        msg = kwds['msg'] if 'msg' in kwds else ''
        print(e, msg, file=sys.stderr)

In [ ]:
"""Test generation of Aer's StateVector with QV """
circ = QuantumVolume(5, seed=1111)
state = AerStatevector(circ)
expected = Statevector(circ)

for e, s in zip(expected, state):
    assertAlmostEqual(e, s)

In [ ]:
"""Test method and device properties"""
circ = QuantumVolume(5, seed=1111)
state1 = AerStatevector(circ)

assertEqual('statevector', state1.metadata()['method'])
assertEqual('CPU', state1.metadata()['device'])

state2 = AerStatevector(circ, method='matrix_product_state')
assertEqual('matrix_product_state', state2.metadata()['method'])
assertEqual('CPU', state2.metadata()['device'])

assertEqual(state1, state2)

In [ ]:
"""Test method and device properties"""
circ1 = QuantumVolume(5, seed=1111)
circ2 = circ1.compose(circ1)
circ3 = circ2.compose(circ1)

state1 = AerStatevector(circ1)
state2 = AerStatevector(circ2)
state3 = AerStatevector(circ3)

assertEqual(state1.evolve(circ1), state2)
assertEqual(state1.evolve(circ1).evolve(circ1), state3)

In [ ]:
"""Test basic gates can be decomposed correctly"""
circ = QuantumCircuit(3)
circ.h(0)
circ.x(1)
circ.ry(np.pi / 2, 2)
state1 = AerStatevector(circ)

In [ ]:
# Test tensor product of 1-qubit gates
circuit = QuantumCircuit(3)
circuit.h(0)
circuit.x(1)
circuit.ry(np.pi / 2, 2)
psi = AerStatevector.from_instruction(circuit)
target = AerStatevector.from_label("000").evolve(Operator(circuit))
assertEqual(target, psi)

In [ ]:
# Test tensor product of 1-qubit gates
circuit = QuantumCircuit(3)
circuit.h(0)
circuit.h(1)
target = AerStatevector.from_label("000").evolve(Operator(circuit))
psi = AerStatevector.from_instruction(circuit)
assertEqual(psi, target)

In [ ]:
"""Test deep copy"""
import copy
circ1 = QuantumVolume(5, seed=1111)

state1 = AerStatevector(circ1)
state2 = copy.deepcopy(state1)

for pa1, pa2 in zip(state1, state2):
    assertAlmostEqual(pa1, pa2)

assertNotEqual(id(state1._data), id(state2._data))

In [ ]:
"""Test ndarray initialization """
circ = QuantumVolume(5, seed=1111)
expected = Statevector(circ)
state = AerStatevector(expected.data)

for e, s in zip(expected, state):
    assertAlmostEqual(e, s)

## Copy from test_statevector.py in terra

In [ ]:
"""Test subsystem initialization from N-qubit array."""
# Test automatic inference of qubit subsystems
vec = rand_vec(8)
for dims in [None, 8]:
    state = AerStatevector(vec, dims=dims)
    assert_allclose(state.data, vec)
    assertEqual(state.dim, 8)
    assertEqual(state.dims(), (2, 2, 2))
    assertEqual(state.num_qubits, 3)

In [ ]:
"""Test subsystem initialization from N-qubit array."""
# Test automatic inference of qubit subsystems
vec = rand_vec(8)
for dims in [None, 8]:
    state = AerStatevector(vec, dims=dims)
    assert_allclose(state.data, vec)
    assertEqual(state.dim, 8)
    assertEqual(state.dims(), (2, 2, 2))
    assertEqual(state.num_qubits, 3)

In [ ]:
"""Test initialization from circuit."""
circuit = QuantumCircuit(3)
circuit.x(0)
state = AerStatevector(circuit)

assertEqual(state.dim, 8)
assertEqual(state.dims(), (2, 2, 2))
assertTrue(all(state.data == np.array([0, 1, 0, 0, 0, 0, 0, 0], dtype=complex)))
assertEqual(state.num_qubits, 3)

In [ ]:
"""Test initialization exception from array."""
vec = rand_vec(4)
assertRaises(QiskitError, AerStatevector, vec, dims=[4, 2])
assertRaises(QiskitError, AerStatevector, vec, dims=[2, 4])
assertRaises(QiskitError, AerStatevector, vec, dims=5)

In [ ]:
"""Test initialization from AerStatevector."""
vec1 = AerStatevector(rand_vec(4))
vec2 = AerStatevector(vec1)
assertEqual(vec1, vec2)

In [ ]:
"""Test initialization from a circuit."""
# random unitaries
u0 = random_unitary(2).data
u1 = random_unitary(2).data
# add to circuit
qr = QuantumRegister(2)
circ = QuantumCircuit(qr)
circ.unitary(u0, [qr[0]])
circ.unitary(u1, [qr[1]])
target = AerStatevector(np.kron(u1, u0).dot([1, 0, 0, 0]))
vec = AerStatevector.from_instruction(circ)
assertEqual(vec, target)

# Test tensor product of 1-qubit gates
circuit = QuantumCircuit(3)
circuit.h(0)
circuit.x(1)
circuit.ry(np.pi / 2, 2)
target = AerStatevector.from_label("000").evolve(Operator(circuit))
psi = AerStatevector.from_instruction(circuit)
assertEqual(psi, target)

# Test decomposition of Controlled-Phase gate
lam = np.pi / 4
circuit = QuantumCircuit(2)
circuit.h(0)
circuit.h(1)
circuit.cp(lam, 0, 1)
target = AerStatevector.from_label("00").evolve(Operator(circuit))
psi = AerStatevector.from_instruction(circuit)
assertEqual(psi, target)

# Test decomposition of controlled-H gate
circuit = QuantumCircuit(2)
circ.x(0)
circuit.ch(0, 1)
target = AerStatevector.from_label("00").evolve(Operator(circuit))
psi = AerStatevector.from_instruction(circuit)
assertEqual(psi, target)

# Test custom controlled gate
qc = QuantumCircuit(2)
qc.x(0)
qc.h(1)
gate = qc.to_gate()
gate_ctrl = gate.control()

circuit = QuantumCircuit(3)
circuit.x(0)
circuit.append(gate_ctrl, range(3))
target = AerStatevector.from_label("000").evolve(Operator(circuit))
psi = AerStatevector.from_instruction(circuit)
assertEqual(psi, target)

# Test initialize instruction
target = AerStatevector([1, 0, 0, 1j]) / np.sqrt(2)
circuit = QuantumCircuit(2)
circuit.initialize(target.data, [0, 1])
psi = AerStatevector.from_instruction(circuit)
assertEqual(psi, target)

# Test reset instruction
target = AerStatevector([1, 0])
circuit = QuantumCircuit(1)
circuit.h(0)
circuit.reset(0)
psi = AerStatevector.from_instruction(circuit)
assertEqual(psi, target)

In [ ]:
"""Test initialization from an instruction."""
target = np.dot(HGate().to_matrix(), [1, 0])
vec = AerStatevector.from_instruction(HGate()).data
global_phase_equivalent = matrix_equal(vec, target, ignore_phase=True)
assertTrue(global_phase_equivalent)

In [ ]:
"""Test initialization from a label"""
x_p = AerStatevector(np.array([1, 1]) / np.sqrt(2))
x_m = AerStatevector(np.array([1, -1]) / np.sqrt(2))
y_p = AerStatevector(np.array([1, 1j]) / np.sqrt(2))
y_m = AerStatevector(np.array([1, -1j]) / np.sqrt(2))
z_p = AerStatevector(np.array([1, 0]))
z_m = AerStatevector(np.array([0, 1]))

label = "01"
target = z_p.tensor(z_m)
assertEqual(target, AerStatevector.from_label(label))

label = "+-"
target = x_p.tensor(x_m)
assertEqual(target, AerStatevector.from_label(label))

label = "rl"
target = y_p.tensor(y_m)
assertEqual(target, AerStatevector.from_label(label))